## CNN sentence classifier - Round 2 of Hyperparameter tuning

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 9.46 ms, sys: 4.31 ms, total: 13.8 ms
Wall time: 12.8 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, tokenizer = tokenize_sentences(x_train, max_len)
x_val_ids, tokenizer = tokenize_sentences(x_val, max_len)

Loading previously created Tokenizer
Loading previously created Tokenizer


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.2517
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim = embed_dim)

Creating new Embedding Matrix
Saved new Embedding Matrix to embedding_matrix.pkl
CPU times: user 27.3 s, sys: 120 ms, total: 27.4 s
Wall time: 27.2 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

8844
8844
2212
2212


In [14]:
%%time

# Each epoch runs in about 2 seconds, so we can use grid search on these models.

run_logfile = "runs2.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [8,16,32], [16,32,64] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            hist = run_model(embedding_matrix,
                                  x_train_ids,
                                  y_train,
                                  x_val_ids,
                                  y_val,
                                  kernel_sizes     = k_sizes,
                                  num_filters      = n_fil,
                                  dense_layer_dims = dld,
                                  epochs = epochs,
                                  logfile = run_logfile)
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
177/177 [==============================] - 2s 8ms/step - loss: 0.5710 - accuracy: 0.7370 - val_loss: 0.4655 - val_accuracy: 0.7884
INFO:tensorflow:Assets written to: ../models/210409-185715/assets
Epoch 2/20
177/177 [==============================] - 1s 6ms/step - loss: 0.4489 - accuracy: 0.7934 - val_loss: 0.4295 - val_accuracy: 0.8151
INFO:tensorflow:Assets written to: ../models/210409-185715/assets
Epoch 3/20
177/177 [==============================] - 1s 6ms/step - loss: 0.3731 - accuracy: 0.8325 - val_loss: 0.3807 - val_accuracy: 0.8368
INFO:tensorflow:Assets written to: ../models/210409-185715/assets
Epoch 4/20
177/177 [==============================] - 1s 6ms/step - loss: 0.2865 - accuracy: 0.8831 - val_loss: 0.3631 - val_accuracy: 0.8400
INFO:tensorflow:Assets written to: ../models/210409-185715/assets
Epoch 5/20
177/177 [==============================] - 1s 6ms/

Epoch 6/20
177/177 [==============================] - 1s 6ms/step - loss: 0.1469 - accuracy: 0.9460 - val_loss: 0.3775 - val_accuracy: 0.8590
INFO:tensorflow:Assets written to: ../models/210409-185806/assets
Epoch 7/20
177/177 [==============================] - 1s 6ms/step - loss: 0.1047 - accuracy: 0.9648 - val_loss: 0.4108 - val_accuracy: 0.8580
Epoch 8/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0761 - accuracy: 0.9716 - val_loss: 0.4479 - val_accuracy: 0.8603
INFO:tensorflow:Assets written to: ../models/210409-185806/assets
Epoch 9/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0762 - accuracy: 0.9722 - val_loss: 0.4756 - val_accuracy: 0.8612
INFO:tensorflow:Assets written to: ../models/210409-185806/assets
Epoch 10/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0452 - accuracy: 0.9866 - val_loss: 0.5145 - val_accuracy: 0.8639
INFO:tensorflow:Assets written to: ../models/210409-185806/assets
Epoch 11/20
177/177 [====

177/177 [==============================] - 1s 6ms/step - loss: 0.0249 - accuracy: 0.9931 - val_loss: 0.5220 - val_accuracy: 0.8766
INFO:tensorflow:Assets written to: ../models/210409-185858/assets
Epoch 13/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0196 - accuracy: 0.9945 - val_loss: 0.5254 - val_accuracy: 0.8698
Epoch 14/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0173 - accuracy: 0.9945 - val_loss: 0.5766 - val_accuracy: 0.8734
Epoch 15/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0169 - accuracy: 0.9948 - val_loss: 0.5914 - val_accuracy: 0.8730
Epoch 16/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0151 - accuracy: 0.9965 - val_loss: 0.6505 - val_accuracy: 0.8698
Epoch 17/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0129 - accuracy: 0.9972 - val_loss: 0.6327 - val_accuracy: 0.8648
Epoch 18/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0101 - accu

Epoch 20/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0162 - accuracy: 0.9958 - val_loss: 0.7897 - val_accuracy: 0.8689



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
177/177 [==============================] - 2s 8ms/step - loss: 0.5446 - accuracy: 0.7505 - val_loss: 0.4642 - val_accuracy: 0.7893
INFO:tensorflow:Assets written to: ../models/210409-190014/assets
Epoch 2/20
177/177 [==============================] - 1s 6ms/step - loss: 0.4532 - accuracy: 0.7939 - val_loss: 0.3989 - val_accuracy: 0.8205
INFO:tensorflow:Assets written to: ../models/210409-190014/assets
Epoch 3/20
177/177 [==============================] - 1s 6ms/step - loss: 0.3526 - accuracy: 0.8410 - val_loss: 0.3692 - val_accuracy: 0.8413
INFO:tensorflow:Assets written to: ../models/210409-190014/assets
Epoch 4/20
177/177 [=========================

177/177 [==============================] - 1s 6ms/step - loss: 0.1645 - accuracy: 0.9398 - val_loss: 0.3681 - val_accuracy: 0.8626
Epoch 6/20
177/177 [==============================] - 1s 6ms/step - loss: 0.1220 - accuracy: 0.9560 - val_loss: 0.3970 - val_accuracy: 0.8535
Epoch 7/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0851 - accuracy: 0.9683 - val_loss: 0.3964 - val_accuracy: 0.8626
Epoch 8/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0521 - accuracy: 0.9855 - val_loss: 0.4606 - val_accuracy: 0.8544
Epoch 9/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0511 - accuracy: 0.9823 - val_loss: 0.4832 - val_accuracy: 0.8671
INFO:tensorflow:Assets written to: ../models/210409-190105/assets
Epoch 10/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0383 - accuracy: 0.9873 - val_loss: 0.5925 - val_accuracy: 0.8698
INFO:tensorflow:Assets written to: ../models/210409-190105/assets
Epoch 11/20
177/177 [=====

...

177/177 [==============================] - 1s 7ms/step - loss: 0.2309 - accuracy: 0.9124 - val_loss: 0.3870 - val_accuracy: 0.8327
Epoch 6/20
177/177 [==============================] - 1s 7ms/step - loss: 0.1803 - accuracy: 0.9318 - val_loss: 0.3728 - val_accuracy: 0.8486
Epoch 7/20
177/177 [==============================] - 1s 7ms/step - loss: 0.1388 - accuracy: 0.9490 - val_loss: 0.3915 - val_accuracy: 0.8508
Epoch 8/20
177/177 [==============================] - 1s 7ms/step - loss: 0.1150 - accuracy: 0.9584 - val_loss: 0.4064 - val_accuracy: 0.8531
INFO:tensorflow:Assets written to: ../models/210409-200635/assets
Epoch 9/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0801 - accuracy: 0.9729 - val_loss: 0.4416 - val_accuracy: 0.8526
Epoch 10/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0681 - accuracy: 0.9759 - val_loss: 0.4920 - val_accuracy: 0.8621
INFO:tensorflow:Assets written to: ../models/210409-200635/assets
Epoch 11/20
177/177 [=====

177/177 [==============================] - 1s 8ms/step - loss: 0.0178 - accuracy: 0.9961 - val_loss: 0.6096 - val_accuracy: 0.8707
Epoch 12/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0120 - accuracy: 0.9970 - val_loss: 0.5794 - val_accuracy: 0.8693
Epoch 13/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0139 - accuracy: 0.9962 - val_loss: 0.7493 - val_accuracy: 0.8648
Epoch 14/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0312 - accuracy: 0.9886 - val_loss: 0.5975 - val_accuracy: 0.8666
Epoch 15/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0132 - accuracy: 0.9966 - val_loss: 0.6230 - val_accuracy: 0.8585
Epoch 16/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0124 - accuracy: 0.9955 - val_loss: 0.6630 - val_accuracy: 0.8684
Epoch 17/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0117 - accuracy: 0.9962 - val_loss: 0.7728 - val_accuracy: 0.8675
Epoch 18/20

177/177 [==============================] - 1s 7ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.8198 - val_accuracy: 0.8684
INFO:tensorflow:Assets written to: ../models/210409-200840/assets
Epoch 18/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0168 - accuracy: 0.9945 - val_loss: 0.8210 - val_accuracy: 0.8671
Epoch 19/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0123 - accuracy: 0.9977 - val_loss: 0.8062 - val_accuracy: 0.8671
Epoch 20/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0088 - accuracy: 0.9974 - val_loss: 0.8477 - val_accuracy: 0.8431



##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 32
Adding Convolution: Kernel Size: 8, Filter Count: 32
Adding Convolution: Kernel Size: 32, Filter Count: 32
Adding Convolution: Kernel Size: 64, Filter Count: 32
Epoch 1/20
177/177 [==============================] - 2s 10ms/step - loss: 0.54

Adding Convolution: Kernel Size: 4, Filter Count: 8
Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 32, Filter Count: 32
Adding Convolution: Kernel Size: 64, Filter Count: 64
Epoch 1/20
177/177 [==============================] - 2s 10ms/step - loss: 0.5915 - accuracy: 0.7324 - val_loss: 0.5065 - val_accuracy: 0.7568
INFO:tensorflow:Assets written to: ../models/210409-201021/assets
Epoch 2/20
177/177 [==============================] - 1s 7ms/step - loss: 0.4869 - accuracy: 0.7487 - val_loss: 0.4336 - val_accuracy: 0.8156
INFO:tensorflow:Assets written to: ../models/210409-201021/assets
Epoch 3/20
177/177 [==============================] - 1s 7ms/step - loss: 0.3893 - accuracy: 0.8305 - val_loss: 0.3488 - val_accuracy: 0.8490
INFO:tensorflow:Assets written to: ../models/210409-201021/assets
Epoch 4/20
177/177 [==============================] - 1s 7ms/step - loss: 0.2833 - accuracy: 0.8781 - val_loss: 0.3426 - val_accuracy: 0.8562
INFO:tensorflow:Asse

177/177 [==============================] - 1s 7ms/step - loss: 0.2684 - accuracy: 0.8890 - val_loss: 0.3865 - val_accuracy: 0.8513
INFO:tensorflow:Assets written to: ../models/210409-201125/assets
Epoch 5/20
177/177 [==============================] - 1s 7ms/step - loss: 0.1819 - accuracy: 0.9321 - val_loss: 0.3610 - val_accuracy: 0.8558
INFO:tensorflow:Assets written to: ../models/210409-201125/assets
Epoch 6/20
177/177 [==============================] - 1s 7ms/step - loss: 0.1250 - accuracy: 0.9557 - val_loss: 0.4329 - val_accuracy: 0.8427
Epoch 7/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0881 - accuracy: 0.9678 - val_loss: 0.4847 - val_accuracy: 0.8590
INFO:tensorflow:Assets written to: ../models/210409-201125/assets
Epoch 8/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0680 - accuracy: 0.9771 - val_loss: 0.5159 - val_accuracy: 0.8540
Epoch 9/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0549 - accuracy: 0.9793 - v





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/